In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

Import dataset and familiar with it

In [24]:
airbnb = pd.read_csv('AB_NYC_2019.csv')
print("Observations in the dataset:",len(airbnb))
airbnb.head()

Observations in the dataset: 48895


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Check type of each column

In [25]:
airbnb.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

To compute the number of 'Nan' values in each column

In [26]:
airbnb.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Drop the irrelevant coloumns: name, host_name (we do not care about their name, and they contain 'Nan' values), last_review (is not helpful).
Append value of reviews_per_month with 0

In [27]:
airbnb = airbnb.drop(['name','host_name','last_review'], axis=1).reset_index(drop=True)
airbnb = airbnb.fillna({'reviews_per_month':0})

Get unique value of 'neighbourhood_group', 'neighbourhood', and 'room_type'

In [32]:
unique_neig_group = airbnb.neighbourhood_group.unique()
unique_neig = airbnb.neighbourhood.unique()
unique_room_type = airbnb.room_type.unique()
print("Unique value of \'neighbor_group\' are:", unique_neig_group)
print("Number of unique value of \'neighbourhood\' is:", len(unique_neig))
print("Unique value of \'room_type\' are:", unique_room_type)

Unique value of 'neighbor_group' are: ['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']
Number of unique value of 'neighbourhood' is: 221
Unique value of 'room_type' are: ['Private room' 'Entire home/apt' 'Shared room']


Analyse the data

In [40]:
# count values of each host_id to get the 10 most popular hosts
top_10_host = airbnb.host_id.value_counts().head(10)
print("the 10 most popular hosts are (just showing the host-id):")
top_10_host

the 10 most popular hosts are (just showing the host-id):


219517861    327
107434423    232
30283594     121
137358866    103
16098958      96
12243051      96
61391963      91
22541573      87
200380610     65
7503643       52
Name: host_id, dtype: int64